In [1]:
################################################
# KNN
################################################

# 1. Exploratory Data Analysis
# 2. Data Preprocessing & Feature Engineering
# 3. Modeling & Prediction
# 4. Model Evaluation
# 5. Hyperparameter Optimization
# 6. Final Model

In [2]:
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [3]:
# 1. Exploratory Data Analysis

In [4]:
df = pd.read_csv("OneDrive/diabetes.csv")

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.shape

(768, 9)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [8]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [9]:
# 2. Data Preprocessing & Feature Engineering

In [10]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [12]:
#kullanacağımız yöntem (KNN yöntemi) uzaklık temelli, uzaklık temelli yöntemlerde değişkenlerin standart olması,
#elde edilecek sonuçların ya daha hızlı ya da daha doğru olmasını sağlayacaktır, diğer bir ifadeyle daha başarılı
#olunmasını sağlayacaktır. bu sebeple elimizdeki bağımsız değişkenleri standartlaştırma işlemlerine sokacağız

X_scaled = StandardScaler().fit_transform(X)

X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [13]:
#çıktıda sütun isimleri yok. ekleyelim

X = pd.DataFrame(X_scaled, columns=X.columns)

In [14]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [15]:
# 3. Modeling & Prediction

In [16]:
knn_model = KNeighborsClassifier().fit(X, y)

#KNeighborsClassifier metodunu başta import etmiştik

In [18]:
random_user = X.sample(1, random_state=45)

#rastgele kullanıcı seçtik random_state kullanımı aynı sonucu almak içindir

In [19]:
knn_model.predict(random_user)

array([1], dtype=int64)

In [20]:
# 4. Model Evaluation

In [21]:
#az önce tek bir tahmin yaptık bütün tahminleri kaydedelim

# Confusion matrix için y_pred:
y_pred = knn_model.predict(X)

In [25]:
#roc eğrisinin altında kalan alanı ifade eden AUC'yi hesplayalım

# AUC için y_prob:
y_prob = knn_model.predict_proba(X)[:, 1]

#1 sınıfına ait olma olasılıklarını getir demiş olduk

In [26]:
print(classification_report(y, y_pred))

#1 sınıfına ve 0 sınıfına göre hesaplama işlemleri yapıyordu
#ana odağımız 1 sınıfı

#accuracy: başarılı sınıflandırma oranı yani 1e 1 dediğimiz 0a 0 dediğimiz durumlar / tüm durumlar = 0.83

#precision: 1 olarak tahmin ettiklerimizin başarısı : 0.79

#recall: gerçekte 1 olanları 1 olarak tahmin etme başarısı : 0.70

#f1-score : precision ve recall'ın harmonik ortalaması

#macro avg : aritmetik ortalama
#weighted avg: ağırlıklı ortalama

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



In [27]:
# AUC : 0.90
roc_auc_score(y, y_prob)

#y: gerçek değerler
#y_prob: y'lerde yer alan 1 sınıfına ait olma olasılıklarını ifade eder

0.9017686567164179

In [28]:
#modeli kurduğumuz veride test etmiş olduk
#yani bütün veriyle bir model kurduk
#kurmuş olduğumuz modelin başarısını yine modeli kurarken kullandığımız veriyle test ettik
#aslında yapılması gereken şey : modelin görmediği verideki performansını değerlendirmek
#bunun için iki yol öğrendik 
#1. Holdout yöntemi , sınama seti yaklaşımı (test, train)
#2. Cross validation yöntemi: holdout yönteminin ortaya çıkarabileceği bazı senaryolardaki dezavantajı gidermek için önerilir

#biz cross validation yöntemi kullanacağız
#örneğin 5 katlı çapraz doğrulama kullanarak hatamızı değerlendireceğiz

In [29]:
# acc 0.83
# f1 0.74

In [30]:
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

#cross_validate metodunu yukarıda import etmiştik 
#cross_validate(model, bağımsız değişken, bağımlı değişken, kaç katlı cv istiyoruz, metrikler)

In [31]:
cv_results

{'fit_time': array([0.00799918, 0.00799966, 0.0060029 , 0.00499606, 0.00512886]),
 'score_time': array([0.03500748, 0.05602121, 0.03062725, 0.02700949, 0.03048992]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [32]:
cv_results['test_accuracy'].mean()

0.733112638994992

In [33]:
cv_results['test_f1'].mean()

0.5905780011534191

In [34]:
cv_results['test_roc_auc'].mean()

0.7805279524807827

In [35]:
#acc %83ten %73e düşmüş
#f1 score %74ten % 59a düşmüş
#auc %90dan %78e düşmüş

#cross validation sonuçları daha güvenilir

In [36]:
#bu başarı skorları nasıl artırılabilir?

# 1. Örnek boyutu arttıralabilir. (yani gözlem sayısı)
# 2. Veri ön işleme işlemleri detaylandırılabilir
# 3. Özellik mühendisliği: yeni değişkenler türetilebilir
# 4. İlgili algoritma için optimizasyonlar yapılabilir.

In [38]:
knn_model.get_params()

#komşuluk sayısı hiperparametresi: 5 miş

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [39]:
#parametre: veri içerisinden öğrenilir

#hiper parametre: kullanıcı tarafından tanımlanması gereken dışsal ve veri seti içerisinden öğrenilemeyen parametrelerdir

In [40]:
# 5. Hyperparameter Optimization

In [41]:
knn_model = KNeighborsClassifier()

#knn model , k en yakın komşu yöntemi

In [43]:
knn_model.get_params()

#bu yöntemin ön tanımlı bir parametresi

#burada komşuluk sayısı 5 

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [45]:
#amaç : bu komşuluk sayısını değiştirerek olması gereken en optimum komşuluk sayısının ne olacağını bulmak

#bunun için şöyle bir parametre listesi oluşturalım

knn_params = {"n_neighbors": range(2, 50)}   

#"n_neighbors" yukarıdaki isimlendirmenin aynısı olması gerekiyor
#range(2, 50) : 2'den 50'ye kadar sayılar oluşturduk
#bu sayıları tek tek deniyor olacağız

In [46]:
#GridSearchCV adında bir yöntem var

#3 4 5 30 48 36 ... komşuluk için knn modeli kurup hatamıza bakacağız

In [48]:
#n_jobs=-1 dendiğinde işlemcileri tam performans ile kullanır

#verbose=1 dendiğinde rapor bekliyorum anlamına geliyor


knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)


#çıktı: denenecek 48 tane hiperparametre aday var diyor, her biri için 5 katlı cross validation yapılacak

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [49]:
knn_gs_best.best_params_

#komşuluk 17 geldi, önceden 5ti, bu 17 ile final modeli kuralım

{'n_neighbors': 17}

In [50]:
# 6. Final Model

#elimizdeki knn algoritmasının en iyi hangi hiperparametre değeri ile çalışabileceğini tespit ettik , bunu bu modele
#göstermeliyiz. daha öncesinde onun ön tanımlı değerini (5) kullandık. şimdi en iyi değeri kullanalım (17)

In [51]:
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

# ** sözlüğü otomatik olarak atadı

In [52]:
cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

In [54]:
cv_results['test_accuracy'].mean()

0.7669892199303965

In [55]:
cv_results['test_f1'].mean()

0.6170909049720137

In [56]:
cv_results['test_roc_auc'].mean()

0.8127938504542278

In [57]:
#3ü de artmış

In [67]:
random_user = X.sample(1)

#bi tane kullanıcı seçtim 
#random state demediğim için aynı sonuç çıkmayacak

random_user

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
308,-1.141852,0.222381,-0.05715,-0.096379,0.870031,-0.189437,2.775863,-0.701198


In [68]:
knn_final.predict(random_user)

array([0], dtype=int64)

In [ ]:
#0 diyabet değil , 1 diyabet